# Building DB of not_100 songs
1. Discogs 3,000 songs
3. Kaggle DB 3,612 songs
4. Spotify 500 songs - with my selected artists/playlists: Joy Crookes, Coldplay, Anoushka Shankar, Lil Simz, Koffee, Parcels, Damian Marley, Protoje, Masego, Eric Clapton, Nitin Sawhney, London Grammar, Ms. Lauryn Hill, Paul Izak, The Beatles, Simon and Garfunkel, Hope Tala, Birdy, Joni Mitchel, FKJ, ABBA, Knxwledge, The Human Experience, Tom Misch, Handmade Moments, Sabrina Starke, Farafi, Jordan Rakei, Jason Mraz, Feist, Paramore, Tracy Chapman, Johann Johannson, Ayọ , A.R. Rahman, Talvin Singh, Buddha's Lounge, Hariprasad Charausia, Hiroshi Yoshimura, Darlini Singh Kaul.

### Discogs Scrape ###

In [1]:
import requests # to download html code
from bs4 import BeautifulSoup # to navigate through the html code
import pandas as pd
import numpy as np
import re  #regexp

In [2]:
one_hot100=pd.read_csv('/Users/devirughani/Desktop/IronHack/Week_6/Spotify_Midterm_Project/hot100.csv')
one_hot100.head()

,song_title,artist
0,Heat Waves,Glass Animals
1,We Don't Talk About Bruno,"Carolina Gaitan, Mauro Castillo, Adassa, Rhenz..."
2,Super Gremlin,Kodak Black
3,abcdefu,GAYLE
4,Stay,The Kid LAROI & Justin Bieber


In [126]:
one_hot100['artist2']=one_hot100['artist'].apply(lambda x: x.partition(',')[0])
one_hot100['main_artist']=one_hot100['artist2'].apply(lambda x: x.partition('&')[0])
one_hot100_clean=one_hot100.drop(columns=['artist', 'artist2'])
one_hot100_clean.to_csv("one_hot100_clean",index=False)

In [4]:
song_titles_100=list(one_hot100_clean['song_title'])
artists_100=list(one_hot100_clean['main_artist'])

In [5]:
url = "https://www.discogs.com/search/?limit=250&sort=have%2Cdesc&type=release&page=1"

In [6]:
# 3. download html with a get request. Use the function request.get() and store the output in response
response = requests.get(url)
# 200 status code means OK! response.status_code
print(response.status_code)

200


In [7]:
# 4.1. parse html (create the 'soup')
soup = BeautifulSoup(response.text, 'html.parser')
# 4.2. check that the html code looks like it should
print(soup.prettify())

<!DOCTYPE html>
<html class="is_not_mobile needs_reduced_ui" lang="en" xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://opengraphprotocol.org/schema/">
 <head>
  <meta charset="utf-8"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta content="en" http-equiv="content-language"/>
  <meta content="no-cache" http-equiv="pragma">
   <meta content="-1" http-equiv="expires">
    <!-- OT will rewrite convert these to javascript and update our consent module accordingly -->
    <script class="optanon-category-C0002" type="text/plain">
     window.consent.resolveGroup(window.consent.PERFORMANCE_GROUP)
    </script>
    <script class="optanon-category-C0003" type="text/plain">
     window.consent.resolveGroup(window.consent.FUNCTIONALITY_GROUP)
    </script>
    <script class="optanon-category-C0004" type="text/plain">
     window.consent.resolveGroup(window.consent.TARGETING_GROUP)
    </script>
    <meta content="initial-scale=1.0,width=device-width" id="viewport" 

In [8]:
#"https://www.discogs.com/search/?limit=250&sort=have%2Cdesc&type=release&page=1
#https://www.discogs.com/search/?limit=250&sort=have%2Cdesc&type=release&page=2
#https://www.discogs.com/search/?limit=250&sort=have%2Cdesc&type=release&page=3

In [9]:
#URL iterations - 8 pages total 

iterations = range(1, 13, 1)

for i in iterations:
    page = str(i)
    url = "https://www.discogs.com/search/?limit=250&sort=have%2Cdesc&type=release&page=" + page 
    print(url)

https://www.discogs.com/search/?limit=250&sort=have%2Cdesc&type=release&page=1
https://www.discogs.com/search/?limit=250&sort=have%2Cdesc&type=release&page=2
https://www.discogs.com/search/?limit=250&sort=have%2Cdesc&type=release&page=3
https://www.discogs.com/search/?limit=250&sort=have%2Cdesc&type=release&page=4
https://www.discogs.com/search/?limit=250&sort=have%2Cdesc&type=release&page=5
https://www.discogs.com/search/?limit=250&sort=have%2Cdesc&type=release&page=6
https://www.discogs.com/search/?limit=250&sort=have%2Cdesc&type=release&page=7
https://www.discogs.com/search/?limit=250&sort=have%2Cdesc&type=release&page=8
https://www.discogs.com/search/?limit=250&sort=have%2Cdesc&type=release&page=9
https://www.discogs.com/search/?limit=250&sort=have%2Cdesc&type=release&page=10
https://www.discogs.com/search/?limit=250&sort=have%2Cdesc&type=release&page=11
https://www.discogs.com/search/?limit=250&sort=have%2Cdesc&type=release&page=12


Respectful scraping:

Before starting with the actual scraping, though, there's something we need to note when sending massive, automated requests to websites: it's rude.

We just have 13 of them, which is not too many, but it's still a good practice to let a few seconds pass in between requests. Some pages don't like being scraped and will block your IP if they detect it's sending automated requests. Others might have a small server for the traffic they handle, and sending too many requests might crash the site.

The sleep module will help us with that.

We will now scrape all the pages and store the response into a list - waiting a few seconds in between requests:

In [10]:
# To make it more "human", we can randomize the waiting time:
from time import sleep
from random import randint

pages = []

for i in range(1, 13, 1):

    # assemble the url:
    page = str(i)
    url = "https://www.discogs.com/search/?limit=250&sort=have%2Cdesc&type=release&page=" + page 

    # download html with a get request:
    response = requests.get(url)

    # monitor the process by printing the status code
    print("Status code: " + str(response.status_code))

    # store response into "pages" list
    pages.append(response)

    # respectful nap:
    wait_time = randint(1,6)
    print("I will sleep for " + str(wait_time) + " second/s.")
    sleep(wait_time)

Status code: 200
I will sleep for 5 second/s.
Status code: 200
I will sleep for 2 second/s.
Status code: 200
I will sleep for 1 second/s.
Status code: 200
I will sleep for 3 second/s.
Status code: 200
I will sleep for 2 second/s.
Status code: 200
I will sleep for 2 second/s.
Status code: 200
I will sleep for 3 second/s.
Status code: 200
I will sleep for 2 second/s.
Status code: 200
I will sleep for 2 second/s.
Status code: 200
I will sleep for 5 second/s.
Status code: 200
I will sleep for 2 second/s.
Status code: 200
I will sleep for 2 second/s.


Note how if you print the object pages after running the code above, you'll just see the response code messages, but the html code is still accessible and you can parse it the same way we've always done:

In [11]:
for i in range(1, 12, 1):
    BeautifulSoup(pages[i].content, "html.parser")

In [12]:
soup_1 = BeautifulSoup(pages[0].content, "html.parser")
soup_2 = BeautifulSoup(pages[1].content, "html.parser")
soup_3 = BeautifulSoup(pages[2].content, "html.parser")
soup_4 = BeautifulSoup(pages[3].content, "html.parser")
soup_5 = BeautifulSoup(pages[4].content, "html.parser")
soup_6 = BeautifulSoup(pages[5].content, "html.parser")
soup_7 = BeautifulSoup(pages[6].content, "html.parser")
soup_8 = BeautifulSoup(pages[7].content, "html.parser")
soup_9 = BeautifulSoup(pages[8].content, "html.parser")
soup_10 = BeautifulSoup(pages[9].content, "html.parser")
soup_11 = BeautifulSoup(pages[10].content, "html.parser")
soup_12 = BeautifulSoup(pages[11].content, "html.parser")

In [13]:
print(soup_1)


<!DOCTYPE html>

<html class="is_not_mobile needs_reduced_ui" lang="en" xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://opengraphprotocol.org/schema/">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="en" http-equiv="content-language"/>
<meta content="no-cache" http-equiv="pragma">
<meta content="-1" http-equiv="expires">
<!-- OT will rewrite convert these to javascript and update our consent module accordingly -->
<script class="optanon-category-C0002" type="text/plain">
            window.consent.resolveGroup(window.consent.PERFORMANCE_GROUP)
        </script>
<script class="optanon-category-C0003" type="text/plain">
            window.consent.resolveGroup(window.consent.FUNCTIONALITY_GROUP)
        </script>
<script class="optanon-category-C0004" type="text/plain">
            window.consent.resolveGroup(window.consent.TARGETING_GROUP)
        </script>
<meta content="initial-scale=1.0,width=device-width" id="viewpor

Locate:
- song name
- artist



#### Song names

In [14]:
soup_1.select("div:nth-child(1)>h4>a")[0].get_text()

'Random Access Memories'

In [15]:
# This is just for one page right now
song_names = []

for div in soup_1.find_all("div", {"class": "card card_large float_fix shortcut_navigable"}):
    for elem in div.find_all("h4"):
        song_names.append(elem.get_text().replace("\n",""))
for div in soup_2.find_all("div", {"class": "card card_large float_fix shortcut_navigable"}):
    for elem in div.find_all("h4"):
        song_names.append(elem.get_text().replace("\n",""))
        
for div in soup_3.find_all("div", {"class": "card card_large float_fix shortcut_navigable"}):
    for elem in div.find_all("h4"):
        song_names.append(elem.get_text().replace("\n",""))
        
for div in soup_4.find_all("div", {"class": "card card_large float_fix shortcut_navigable"}):
    for elem in div.find_all("h4"):
        song_names.append(elem.get_text().replace("\n",""))
        
for div in soup_5.find_all("div", {"class": "card card_large float_fix shortcut_navigable"}):
    for elem in div.find_all("h4"):
        song_names.append(elem.get_text().replace("\n",""))
        
for div in soup_6.find_all("div", {"class": "card card_large float_fix shortcut_navigable"}):
    for elem in div.find_all("h4"):
        song_names.append(elem.get_text().replace("\n",""))
        
for div in soup_7.find_all("div", {"class": "card card_large float_fix shortcut_navigable"}):
    for elem in div.find_all("h4"):
        song_names.append(elem.get_text().replace("\n",""))
        
for div in soup_8.find_all("div", {"class": "card card_large float_fix shortcut_navigable"}):
    for elem in div.find_all("h4"):
        song_names.append(elem.get_text().replace("\n",""))
        
for div in soup_9.find_all("div", {"class": "card card_large float_fix shortcut_navigable"}):
    for elem in div.find_all("h4"):
        song_names.append(elem.get_text().replace("\n",""))
        
for div in soup_10.find_all("div", {"class": "card card_large float_fix shortcut_navigable"}):
    for elem in div.find_all("h4"):
        song_names.append(elem.get_text().replace("\n",""))
        
for div in soup_11.find_all("div", {"class": "card card_large float_fix shortcut_navigable"}):
    for elem in div.find_all("h4"):
        song_names.append(elem.get_text().replace("\n",""))
        
for div in soup_12.find_all("div", {"class": "card card_large float_fix shortcut_navigable"}):
    for elem in div.find_all("h4"):
        song_names.append(elem.get_text().replace("\n",""))
        
len(song_names)

3000

#### Artist Names

In [16]:
# This is just for one page right now
artists = []
for div in soup_1.find_all("div", {"class": "card card_large float_fix shortcut_navigable"}):
    for elem in div.find_all("h5"):
        artists.append(elem.get_text().replace("\n",""))
for div in soup_2.find_all("div", {"class": "card card_large float_fix shortcut_navigable"}):
    for elem in div.find_all("h5"):
        artists.append(elem.get_text().replace("\n",""))
for div in soup_3.find_all("div", {"class": "card card_large float_fix shortcut_navigable"}):
    for elem in div.find_all("h5"):
        artists.append(elem.get_text().replace("\n",""))
for div in soup_4.find_all("div", {"class": "card card_large float_fix shortcut_navigable"}):
    for elem in div.find_all("h5"):
        artists.append(elem.get_text().replace("\n",""))
for div in soup_5.find_all("div", {"class": "card card_large float_fix shortcut_navigable"}):
    for elem in div.find_all("h5"):
        artists.append(elem.get_text().replace("\n",""))
for div in soup_6.find_all("div", {"class": "card card_large float_fix shortcut_navigable"}):
    for elem in div.find_all("h5"):
        artists.append(elem.get_text().replace("\n",""))
for div in soup_7.find_all("div", {"class": "card card_large float_fix shortcut_navigable"}):
    for elem in div.find_all("h5"):
        artists.append(elem.get_text().replace("\n",""))
for div in soup_8.find_all("div", {"class": "card card_large float_fix shortcut_navigable"}):
    for elem in div.find_all("h5"):
        artists.append(elem.get_text().replace("\n",""))
for div in soup_9.find_all("div", {"class": "card card_large float_fix shortcut_navigable"}):
    for elem in div.find_all("h5"):
        artists.append(elem.get_text().replace("\n",""))
for div in soup_10.find_all("div", {"class": "card card_large float_fix shortcut_navigable"}):
    for elem in div.find_all("h5"):
        artists.append(elem.get_text().replace("\n",""))
for div in soup_11.find_all("div", {"class": "card card_large float_fix shortcut_navigable"}):
    for elem in div.find_all("h5"):
        artists.append(elem.get_text().replace("\n",""))
for div in soup_12.find_all("div", {"class": "card card_large float_fix shortcut_navigable"}):
    for elem in div.find_all("h5"):
        artists.append(elem.get_text().replace("\n",""))
    
len(artists)

3000

In [18]:
pages_parsed = []
song_names = []
artists = []

for i in range(len(pages)):
    # parse all pages
    pages_parsed.append(BeautifulSoup(pages[i].content, "html.parser"))
    # select only the info about the movies
    #discogs_html = pages_parsed[i]
    for div in soup_1.find_all("div", {"class": "card card_large float_fix shortcut_navigable"}):
        for elem in div.find_all("h4"):
            song_names.append(elem.get_text().replace("\n",""))
        for elem2 in div.find_all("h5"):
            artists.append(elem2.get_text().replace("\n",""))
        
    for div in soup_2.find_all("div", {"class": "card card_large float_fix shortcut_navigable"}):
        for elem in div.find_all("h4"):
            song_names.append(elem.get_text().replace("\n",""))
        for elem2 in div.find_all("h5"):
            artists.append(elem2.get_text().replace("\n",""))
    
    for div in soup_3.find_all("div", {"class": "card card_large float_fix shortcut_navigable"}):
        for elem in div.find_all("h4"):
            song_names.append(elem.get_text().replace("\n",""))
        for elem in div.find_all("h5"):
            artists.append(elem.get_text().replace("\n",""))
        
    for div in soup_4.find_all("div", {"class": "card card_large float_fix shortcut_navigable"}):
        for elem in div.find_all("h4"):
            song_names.append(elem.get_text().replace("\n",""))
        for elem2 in div.find_all("h5"):
            artists.append(elem2.get_text().replace("\n",""))
    
    for div in soup_5.find_all("div", {"class": "card card_large float_fix shortcut_navigable"}):
        for elem in div.find_all("h4"):
            song_names.append(elem.get_text().replace("\n",""))
        for elem2 in div.find_all("h5"):
            artists.append(elem2.get_text().replace("\n",""))
    
    for div in soup_6.find_all("div", {"class": "card card_large float_fix shortcut_navigable"}):
        for elem in div.find_all("h4"):
            song_names.append(elem.get_text().replace("\n",""))
        for elem2 in div.find_all("h5"):
            artists.append(elem2.get_text().replace("\n",""))
    
    for div in soup_7.find_all("div", {"class": "card card_large float_fix shortcut_navigable"}):
        for elem in div.find_all("h4"):
            song_names.append(elem.get_text().replace("\n",""))
        for elem2 in div.find_all("h5"):
            artists.append(elem2.get_text().replace("\n",""))  
    
    for div in soup_8.find_all("div", {"class": "card card_large float_fix shortcut_navigable"}):
        for elem in div.find_all("h4"):
            song_names.append(elem.get_text().replace("\n",""))
        for elem2 in div.find_all("h5"):
            artists.append(elem2.get_text().replace("\n",""))
    
    for div in soup_9.find_all("div", {"class": "card card_large float_fix shortcut_navigable"}):
        for elem in div.find_all("h4"):
            song_names.append(elem.get_text().replace("\n",""))
        for elem2 in div.find_all("h5"):
            artists.append(elem2.get_text().replace("\n",""))
    
    for div in soup_10.find_all("div", {"class": "card card_large float_fix shortcut_navigable"}):
        for elem in div.find_all("h4"):
            song_names.append(elem.get_text().replace("\n",""))
        for elem2 in div.find_all("h5"):
            artists.append(elem2.get_text().replace("\n",""))
    
    for div in soup_11.find_all("div", {"class": "card card_large float_fix shortcut_navigable"}):
        for elem in div.find_all("h4"):
            song_names.append(elem.get_text().replace("\n",""))
        for elem2 in div.find_all("h5"):
            artists.append(elem2.get_text().replace("\n",""))
    
    for div in soup_12.find_all("div", {"class": "card card_large float_fix shortcut_navigable"}):
        for elem in div.find_all("h4"):
            song_names.append(elem.get_text().replace("\n",""))
        for elem2 in div.find_all("h5"):
            artists.append(elem2.get_text().replace("\n",""))
# Checking our output:
print(len(song_names)) # output: 
print(len(artists))  # output: 

#song_names.drop_duplicates()
#artists.drop_duplicates()

print(len(song_names)) # output: 
print(len(artists))  # output: 

36000
36000


AttributeError: 'list' object has no attribute 'drop_duplicates'

In [19]:
discogs_not_hot_songs = pd.DataFrame({"song_title":song_names,
                           "artist":artists})
discogs_not_hot_songs

,song_title,artist
0,Random Access Memories,Daft Punk
1,"Good Kid, M.A.A.d City",Kendrick Lamar
2,Thriller,Michael Jackson
3,The Dark Side Of The Moon,Pink Floyd
4,Rumours,Fleetwood Mac
...,...,...
35995,Sailing The Seas Of Cheese,Primus
35996,The B-52's,The B-52's
35997,Off The Wall,Michael Jackson
35998,Shaken 'N' Stirred,Robert Plant


In [83]:
discogs_not_hot_songs=discogs_not_hot_songs.drop_duplicates()
discogs_not_hot_songs.to_csv("discogs_not_hot_songs.csv", index = False)

In [21]:
!ls

Web_scraping_Lab1.ipynb               hot100.csv
__pycache__                           not_hot_100_notebook.ipynb
case-study-gnod.md                    not_onehot_songs.csv
config.py                             tracks_features.csv
discogs_not_hot_songs.csv             using_spotipy_project_continued.ipynb


### Kaggle Music DB ###

In [22]:
kaggle_database=pd.read_csv('/Users/devirughani/Desktop/IronHack/Week_6/Spotify_Midterm_Project/tracks_features.csv')
kaggle_database.head() #care about columns 'name' and 'artists'
kaggle_not_hot_songs=kaggle_database.sample(frac=0.003, replace=False, random_state=1)
kaggle_not_hot_songs=kaggle_not_hot_songs[['name','artists']]

kaggle_not_hot_songs['artists'] = kaggle_not_hot_songs['artists'].apply(lambda x: x.replace("[", "").replace("]", "").replace("'", ""))
kaggle_not_hot_songs.reset_index(inplace=True)
kaggle_not_hot_songs.head(10)
print(kaggle_not_hot_songs.shape)

(3612, 3)


In [23]:
kaggle_not_hot_songs=kaggle_not_hot_songs.drop(columns=['index'])

In [24]:
kaggle_not_hot_songs=kaggle_not_hot_songs.rename(columns={"name": "song_title", "artists": "artist"})

### Merging Discogs and Kaggle Output ###

In [25]:
#Concat2 dataframes
not_onehot_songs=pd.concat([discogs_not_hot_songs, kaggle_not_hot_songs])
not_onehot_songs.shape

(39612, 2)

In [86]:
# Check for duplicates
not_onehot_songs_new = not_onehot_songs.drop_duplicates()
not_onehot_songs_new.shape

(5675, 2)

In [90]:
#Save them into a csv at the end 
not_onehot_songs.to_csv("not_onehot_songs.csv",index=False)
not_onehot_songs_new.to_csv("not_onehot_songs_new.csv",index=False)

In [91]:
!ls

Web_scraping_Lab1.ipynb                hot100.csv
__pycache__                            not_hot_100_notebook.ipynb
audio_feats_one_hot100_songs.pkl       not_onehot_songs.csv
audio_features_one_hot100_songs_df.csv not_onehot_songs_new.csv
case-study-gnod.md                     one_hot100_songs_ids.pkl
config.py                              tracks_features.csv
discogs_not_hot_songs.csv              using_spotipy_project_continued.ipynb


### Spotipy ###


In [ ]:
#!pip install spotipy

In [1]:
import sys
sys.path.insert(1,'/Users/devirughani/Desktop/IronHack/Week_6/Spotify_Midterm_Project/config.py')
from config import *

In [30]:
import spotipy
import json
from spotipy.oauth2 import SpotifyClientCredentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= client_id,
                                                           client_secret= client_secret_id))


In [31]:
results = sp.search(q= "track:"'Halo'" artist:"'Beyoncé', limit=5)
results['tracks']['items']
#song = sp.search(q="track:"+song_name+" artist:"+artist_name,limit=5)

[]

In [32]:
tests = sp.search(q ='Halo', limit=6)
tests['tracks'].keys()
tests['tracks']['items'][0].keys()
tests['tracks']['items'][0]['artists'][0]['name']
#tests['tracks']['items'][0]['name']
#tests['tracks']['items'][0]['id']
#for item in tests['tracks']['items']:
    #print("The name of song is: '{}' by:'{}' and the link is: {}".format(item['name'],item['artists'][0]['name'],item['external_urls']['spotify']))

'Beyoncé'

1. Create a function to search a given song in the Spotify API: search_song(). Take into account that sometimes Spotify's API will return several matches for the same song title (different artists, a different album of the same artist, version of the song,...etc). Then it will be nice to display a list of outputs to the user and let him/her select which is the right match. Once the desired song is located, the function should return the href/id/uri of the song to the code (not to the user).

In [33]:
def search_song():
    '''
    Assuming that we have all the libraries imported, this function prompts the user for the song name, 
    after which it provide a list of 10 possible songs. Then the user has to pick the number of the song that they
    want. And based on this it will return the href, track_id, and uri of that song.
    '''

    print('Enter song name:')
    song = input() #Input asking for song name
    
    song_options = sp.search(q= song,limit=10)
    i=0
    for item in song_options['tracks']['items']:#for loop that displays song name, the artist and url 
        print(i,".The name of song is: '{}' by '{}' and the url is: {}".format(item['name'],item['artists'][0]['name'], item['external_urls']['spotify']))
        i+=1
    print('Enter song number:')
    song_selection  = input()#input which asks for the number of the song(ordered number)
    song_selection =int(song_selection)
   
    track_id = song_options['tracks']['items'][song_selection]['id']
    
    return track_id

In [ ]:
search_song()

In [ ]:
##def search_song(song_titles):
       #song_options= sp.search(q=song_title, limit=6) search(q="track:"+song_name+" artist:"+artist_name,)
 #       song_options = sp.search(q= song_titles, limit = 5)
 #       for index, song in enumerate(song_options['tracks']['items']):
  #          print("The song title is: '{}' by:'{}'".format(index['name'],index['artists'][0]['name']))
   #  
   #     print('Enter song number you want to play:')# This will be an string 0, '0'
   #     song_selection = input()
  #      song_selection = int(input())
   #     print("Here is the link to the song you want to play: '{}' song name:'{}' by: {}".format(item['external_urls']['spotify'], item['name'],item['artists'][song_selection]['name']))
   #     track_id = song_options['tracks']['items'][song_selection]['id']
    #    return track_id

In [ ]:
#def search_song2(): 
   #  = sq.search(song, limit=1)
#print('Enter song name:')
#    x = input() #Input asking for song name
    
#    results = sp.search(q=x,limit=10)
 #   i=0
  #  for item in results['tracks']['items']:#for loop that displays song name, the artist and url 
   #     print(i,".The name of song is: '{}' by '{}' and the url is: {}".format(item['name'],item['artists'][0]['name'], item['external_urls']['spotify']))
    #    i+=1
    #print('Enter song number:')
    #a = input()#input which asks for the number of the song(ordered number)
    #a= int(a)
    #track_id = results['tracks']['items'][a]['id']
 
   # return href, track_id, uri

In [34]:
one_hot100_ids=search_song_ids(song_titles_100)
len(one_hot100_ids)

NameError: name 'search_song_ids' is not defined

In [35]:
def search_song_ids(list_of_songs):
    '''
Add explanation/description

    '''
    list_of_song_ids = []
    
    for i in list_of_songs:
        song = sp.search(q= i,limit=1)
        song_ids.append(song["tracks"]["items"][0]["id"])
        list_of_song_ids.append(song['tracks']['items'][0]['id'])
        
    return list_of_song_ids

In [36]:
search_song_ids(song_titles_100)

NameError: name 'song_ids' is not defined

In [37]:
def search_song_ids_improved2(df):
 
    list_of_song_ids = []
    
    for index in range(df.shape[0]): # range(5) -> 0,1,2,3,4
        songs = sp.search(q="track:"+df.iloc[index,0],limit=10)
        #songs = sp.search(q="track:"+df.iloc[index,0]+"artist:"+df.iloc[index,1],limit=2
        # [elem['artists'][0]['name'].lower() for elem in test2['tracks']['items']]
        #print("The content of songs['tracks']['items'] is: ",songs['tracks']['items'])
        #test2['tracks']['items'][-1]['id'] # song id :
        if ( (songs != None) and (len(songs['tracks']['items']) > 0)):
            list_of_song_ids.append(songs['tracks']['items'][-1]['id'])
        else:
            list_of_song_ids.append("")        
        
    return list_of_song_ids

In [ ]:
def search_song_ids_improved2(df):
 
    list_of_song_ids = []
    
    for index in range(df.shape[0]): # range(5) -> 0,1,2,3,4
        songs = sp.search(q="track:"+df.iloc[index,0],limit=10)
        #songs = sp.search(q="track:"+df.iloc[index,0]+"artist:"+df.iloc[index,1],limit=2
        # [elem['artists'][0]['name'].lower() for elem in test2['tracks']['items']]
        #print("The content of songs['tracks']['items'] is: ",songs['tracks']['items'])
        #test2['tracks']['items'][-1]['id'] # song id :
        if ( (songs != None) and (len(songs['tracks']['items']) > 0)):
            list_of_song_ids.append(songs['tracks']['items'][-1]['id'])
        else:
            list_of_song_ids.append("")        
        
    return list_of_song_ids

In [ ]:
len([])

In [38]:
temp2=search_song_ids_improved2(one_hot100_clean)

In [39]:
# https://open.spotify.com/track/0MT0SFNlYTFEBjMYc19qqB
temp2

['0MT0SFNlYTFEBjMYc19qqB',
 '0bE2hfiLQiuJDWm7rtaEWh',
 '5Znse8mm5nD6tmZTNr5hCw',
 '09tU4Rj3VeiFNPQ9eqJCG0',
 '5ubvP9oKmxLUVq506fgLhk',
 '53sI71bFwGU3p2RBeRxWcC',
 '19LvG1Uqsk9KyRRYhLC8Ib',
 '35MPKqnvJFDaXYmvJJ7DoZ',
 '3EfJK2rJtMjGZoIdBrFqh6',
 '1fD9pWBau5bYqsohqa9E8e',
 '02pjWffN5Npwv22BShj1vu',
 '5hx04lWeHrBazQAvu2aLNM',
 '26TpLkciA8vz3kAvSI5tHG',
 '560kOKHQH5oLcnhvMdfExX',
 '5RgFlk1fcClZd0Y4SGYhqH',
 '6eqWA6blXyJWkt2BjZdX8d',
 '5OtNHOdnTdfMALthM3bJQN',
 '2jPQGsL5ZuIp9NTlvZ4TKD',
 '0ypcpkYOckOLxySl4m69gM',
 '5RGFmUJJExsOwN8pClIH87',
 '5yhxyeVnInMpspAq1LZ6KC',
 '62XoIh6od0W79r8Tk51VOp',
 '4EveU9Zb50mjgi5avDNqlK',
 '5iOVjqofxMbfMdc3zuXFnE',
 '09ywFRtU1Ua5M4eO00AFGl',
 '1kzbVP89SVWZY2j5WFnzz2',
 '6r0uIB00z97uxWOlk0vUhN',
 '2LBnn1xStkbkEYVvN0SQQR',
 '7gMOe0gXYcELUoVugfMmHP',
 '76VQNrW5hfgf6GfiPieyEU',
 '5dkTQlAVdA49mAXQSS5P9t',
 '6bF9cN8I9isRETwTYKrCfR',
 '1ELMTB7UyrWWxigLSc366g',
 '4YViBGKSgDuFpVgaZfLlkm',
 '28ELJy93qd1T88JZMXo9LQ',
 '0yiKrbvBZhGtIMqAYgq7pe',
 '7ArGJFYaZrKRQep2cZCRRC',
 

In [41]:
import pickle
with open("one_hot100_songs_ids.pkl","wb") as file:
    pickle.dump(temp2,file)

In [42]:
!ls

Web_scraping_Lab1.ipynb               not_hot_100_notebook.ipynb
__pycache__                           not_onehot_songs.csv
case-study-gnod.md                    one_hot100_songs_ids.pkl
config.py                             tracks_features.csv
discogs_not_hot_songs.csv             using_spotipy_project_continued.ipynb
hot100.csv


In [ ]:
len(list(np.argwhere(np.array(temp) != "")))

In [ ]:
len(list(np.argwhere(np.array(temp2) != "")))

In [ ]:
one_hot100.head()

In [ ]:
one_hot100_clean.head()

In [ ]:
#test = sp.search(q="track:"+"Heat Waves"+"artist:"+"Glass Animals",limit=2)
test2 = sp.search(q="track:"+one_hot100_clean.iloc[index,0],limit=5)
test2

In [ ]:
#{'spotify': 'https://open.spotify.com/track/6CDzDgIUqeDY5g8ujExx2f'}
# https://open.spotify.com/track/3USxtqRwSYz57Ewm6wWRMp
# https://open.spotify.com/track/64QbeyqzpKeU0QZeiqQ6gD
# 6CDzDgIUqeDY5g8ujExx2f
test2['tracks']['items'][-1]['id']


In [ ]:
test2['tracks']['items'][-1]['artists'][0]['name']

In [ ]:
[elem['artists'][0]['name'].lower() for elem in test2['tracks']['items']]

In [ ]:
test

In [168]:
artists = ['tracks']['items'][i]['artists']

<>:1: SyntaxWarning: list indices must be integers or slices, not str; perhaps you missed a comma?
<>:1: SyntaxWarning: list indices must be integers or slices, not str; perhaps you missed a comma?
/var/folders/mw/6447zlmn5h9_r3fs75_lzjgm0000gn/T/ipykernel_84631/1576377252.py:1: SyntaxWarning: list indices must be integers or slices, not str; perhaps you missed a comma?
  artists = ['tracks']['items'][i]['artists']


TypeError: list indices must be integers or slices, not str

In [ ]:
song_ids = []
for index in range(one_hot100_clean.shape[0]):
        #songs = sp.search(q=df.iloc[index,0], limit=5)#search for the song by its title
        songs = sp.search(q="track:"+one_hot100_clean.iloc[index,0]+"artist:"+one_hot100_clean.iloc[index,1],limit=5) # This returns None if the song is not found
        if (songs != None): 
            artists = [songs['tracks']['items'][i]['artists'] for i in range(len(songs))]
            artist_loc = artists.i(one_hot100_clean.iloc[index,1]) if one_hot100_clean.iloc[index,1] in artists else -1
            if (artist_loc != -1 ):
                song_ids.append(song['tracks']['items'][artist_loc]['id'])
            else:
                song_ids.append("")
        else:
            song_ids.append("")            

2. Create a function "get_audio_features(list_of_songs)" to obtain the audio features of a given list of songs (the content of list_of_songs can be the href/id/uri). Then, use this function to create a Pandas Dataframe with the audio features of the list of songs. Hint: create a dictionary with the song's audio features as keys and an empty list as values. Then fill in the lists with the corresponding audio features of each song. Finally, create your data frame from the dictionary. Bear in mind the following: This API has a restriction on the number of requests per minute, therefore, consider launching the search using smaller groups of songs.

In [100]:
not_onehot_100=pd.read_csv('/Users/devirughani/Desktop/IronHack/Week_6/Spotify_Midterm_Project/data/not_onehot_songs_new.csv')

In [101]:
not_onehot_100.head()

,song_title,artist
0,Random Access Memories,Daft Punk
1,"Good Kid, M.A.A.d City",Kendrick Lamar
2,Thriller,Michael Jackson
3,The Dark Side Of The Moon,Pink Floyd
4,Rumours,Fleetwood Mac


In [105]:
not_onehot_100['artist2']=not_onehot_100['artist'].apply(lambda x: x.partition(',')[0])
not_onehot_100.head(20)
not_onehot_100_clean=not_onehot_100.drop(columns=['artist'])
not_onehot_100_clean.to_csv('audio_features_one_hot100_songs_df.csv",index=False')

In [169]:
not_onehot_100_clean.head()

,song_title,artist2
0,Random Access Memories,Daft Punk
1,"Good Kid, M.A.A.d City",Kendrick Lamar
2,Thriller,Michael Jackson
3,The Dark Side Of The Moon,Pink Floyd
4,Rumours,Fleetwood Mac


In [51]:
def get_audio_features(df):
    import math
    import pickle
    import pandas as pd
    import sys
    import spotipy
    from spotipy.oauth2 import SpotifyClientCredentials
    from time import sleep
    
    song_ids = []

    black = {'danceability': "Null", 
           'energy': "Null", 
           'key': "Null",
           'loudness': "Null", 
           'mode':"Null", 
           'speechiness': "Null", 
           'acousticness': "Null", 
           'instrumentalness': "Null", 
            'liveness': "Null", 
            'valence': "Null", 
            'tempo': "Null", 
            'type': "Null", 
            'id': "Null", 
            'uri': "Null", 
            'track_href': "Null", 
            'analysis_url': "Null", 
            'duration_ms': "Null", 
            'time_signature': "Null"}
    
    keys = list(black.keys())
    
    #df2 = pd.DataFrame()
    chunks = math.ceil(df.shape[0]/1000)
    for i in range(chunks): # chunks = 4 -> 0,1,2,3
        song_ids = []
        if ( i < chunks-1 ):
            j = i + 1
        else:
            j = len(df)
        for index, song in enumerate(df.iloc[1000*i:1000*j,0]): #[0:1000],[1000:2000],[2000:3000],[3000:]
            print("Looking for song: ",index)
            try:
                songs = sp.search(q=song, limit=1)
                song_ids.append(songs['tracks']['items'][-1]['id'])
            except: # Si no esta no se para.
                print("The {} has not been found",format(song))
                song_ids.append("")
    k
        print("Getting audio features")
        audio_feats = [sp.audio_features(song_id)[0] if ((song_id != None) and ( song_id != "")) else black for song_id in song_ids]
        name = "audio_feats_" + str(i) + ".pickle"
        with open(name, "wb") as handle:
            pickle.dump(audio_feats,handle)
        print("Created file: ",name)
        sleep(10)
        print('Sleeping for 10 seconds')
    

In [53]:
get_audio_features(one_hot100_clean)

Looking for song:  0
Looking for song:  1
Looking for song:  2
Looking for song:  3
Looking for song:  4
Looking for song:  5
Looking for song:  6
Looking for song:  7
Looking for song:  8
Looking for song:  9
Looking for song:  10
Looking for song:  11
Looking for song:  12
Looking for song:  13
Looking for song:  14
Looking for song:  15
Looking for song:  16
Looking for song:  17
Looking for song:  18
Looking for song:  19
Looking for song:  20
Looking for song:  21
Looking for song:  22
Looking for song:  23
Looking for song:  24
Looking for song:  25
Looking for song:  26
Looking for song:  27
Looking for song:  28
Looking for song:  29
Looking for song:  30
Looking for song:  31
Looking for song:  32
Looking for song:  33
Looking for song:  34
Looking for song:  35
Looking for song:  36
Looking for song:  37
Looking for song:  38
Looking for song:  39
Looking for song:  40
Looking for song:  41
Looking for song:  42
Looking for song:  43
Looking for song:  44
Looking for song:  4

In [59]:
audio_features_one_hot100_songs_df.to_csv("audio_features_one_hot100_songs_df.csv",index=False)

In [54]:
!ls audio*

audio_feats_0.pickle


In [55]:
!mv audio_feats_0.pickle audio_feats_one_hot100_songs.pickle

In [56]:
!ls


Web_scraping_Lab1.ipynb               hot100.csv
__pycache__                           not_hot_100_notebook.ipynb
audio_feats_one_hot100_songs.pickle   not_onehot_songs.csv
case-study-gnod.md                    one_hot100_songs_ids.pkl
config.py                             tracks_features.csv
discogs_not_hot_songs.csv             using_spotipy_project_continued.ipynb


In [62]:
!ls *.pickle

audio_feats_one_hot100_songs.pickle


In [63]:
!mv audio_feats_one_hot100_songs.pickle audio_feats_one_hot100_songs.pkl

3. Once the previous function has been created, create another function "add_audio_features(df, audio_features_df)" to concat a given data frame with the data frame containing the audio features alongside any other desired info, and return the extended data frame.

In [165]:
#this sorts the pkl files in order:
import pickle
import glob

pkls = glob.glob("audio_feats_*.pickle")
pkls.sort()
pkls

#this converts the files into dataframes:)
final_df = pd.DataFrame()
for i, pkl in enumerate(pkls):
   # try:
        print(pkl)
        with open(pkl, "rb") as handle:
            audio_feats = pickle.load(handle)
            audio_feats_df = pd.DataFrame(audio_feats)
            final_df = pd.concat([final_df,audio_feats_df],axis=0).reset_index(drop = True)

    #except:
        #print("Corrupted",pkl)
      #  continue
        
final_df

audio_feats_0.pickle
audio_feats_1.pickle
audio_feats_2.pickle
audio_feats_3.pickle


AttributeError: 'NoneType' object has no attribute 'keys'

In [141]:
final_df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.737,0.399,0,-6.337,1,0.305,0.171,0,0.214,0.438,128.019,audio_features,3c1maqNp7lSVWUeOacgVnh,spotify:track:3c1maqNp7lSVWUeOacgVnh,https://api.spotify.com/v1/tracks/3c1maqNp7lSV...,https://api.spotify.com/v1/audio-analysis/3c1m...,244176,4
1,0.451,0.831,5,-8.641,0,0.307,0.0623,0,0.185,0.41,176.162,audio_features,5vuJuBqwzHJgCA1ysRfwxZ,spotify:track:5vuJuBqwzHJgCA1ysRfwxZ,https://api.spotify.com/v1/tracks/5vuJuBqwzHJg...,https://api.spotify.com/v1/audio-analysis/5vuJ...,214120,4
2,0.773,0.859,11,-4.913,1,0.0747,0.0855,0.00018,0.914,0.813,118.458,audio_features,3S2R0EVwBSAVMd5UMgKTL0,spotify:track:3S2R0EVwBSAVMd5UMgKTL0,https://api.spotify.com/v1/tracks/3S2R0EVwBSAV...,https://api.spotify.com/v1/audio-analysis/3S2R...,357267,4
3,0.238,0.943,9,-6.308,1,0.13,0.000449,0.000004,0.102,0.181,113.419,audio_features,4dxT3byfL0g1mlPjHxk18d,spotify:track:4dxT3byfL0g1mlPjHxk18d,https://api.spotify.com/v1/tracks/4dxT3byfL0g1...,https://api.spotify.com/v1/audio-analysis/4dxT...,248613,4
4,0.844,0.262,0,-14.08,1,0.0899,0.68,0,0.111,0.274,104.945,audio_features,4PKZmIXiIt54OQo1E51aTd,spotify:track:4PKZmIXiIt54OQo1E51aTd,https://api.spotify.com/v1/tracks/4PKZmIXiIt54...,https://api.spotify.com/v1/audio-analysis/4PKZ...,151110,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,0.839,0.601,9,-7.608,1,0.25,0.051,0.000223,0.0817,0.479,101.995,audio_features,6UGht8GPlOk1tIdxBvQLY0,spotify:track:6UGht8GPlOk1tIdxBvQLY0,https://api.spotify.com/v1/tracks/6UGht8GPlOk1...,https://api.spotify.com/v1/audio-analysis/6UGh...,200333,4
2996,0.581,0.656,10,-11.248,1,0.0505,0.254,0,0.215,0.376,136.006,audio_features,3uFwBt6PifAgABEIpcAFcJ,spotify:track:3uFwBt6PifAgABEIpcAFcJ,https://api.spotify.com/v1/tracks/3uFwBt6PifAg...,https://api.spotify.com/v1/audio-analysis/3uFw...,244707,4
2997,0.769,0.638,11,-5.054,1,0.216,0.0902,0,0.0857,0.782,115.007,audio_features,2EvwLVrnYbCZEG6Kx5DCRy,spotify:track:2EvwLVrnYbCZEG6Kx5DCRy,https://api.spotify.com/v1/tracks/2EvwLVrnYbCZ...,https://api.spotify.com/v1/audio-analysis/2Evw...,212107,4
2998,0.546,0.828,4,-4.771,1,0.0394,0.719,0.03,0.372,0.401,121.418,audio_features,5i7rT8lbGzjj1n7TTXR5U8,spotify:track:5i7rT8lbGzjj1n7TTXR5U8,https://api.spotify.com/v1/tracks/5i7rT8lbGzjj...,https://api.spotify.com/v1/audio-analysis/5i7r...,153947,4


In [136]:
!ls

K-Means Clustering.ipynb
Web_scraping_Lab1.ipynb
__pycache__
audio_feats_0.pickle
audio_feats_1.pickle
audio_feats_2.pickle
audio_feats_3.pickle
audio_feats_4.pickle
audio_feats_5.pickle
audio_feats_one_hot100_songs.pkl
audio_features_one_hot100_songs_df.csv
audio_features_one_hot100_songs_df.csv",index=False
case-study-gnod.md
config.py
discogs_not_hot_songs.csv
hot100.csv
hot_songs_and_audio_features.csv
not_hot_100_notebook.ipynb
not_onehot_songs.csv
not_onehot_songs_new.csv
one_hot100_clean
one_hot100_songs_ids.pkl
tracks_features.csv
using_spotipy_project_continued.ipynb


In [150]:
not_onehot_songs_new.reset_index(drop=True)

,song_title,artist
0,Random Access Memories,Daft Punk
1,"Good Kid, M.A.A.d City",Kendrick Lamar
2,Thriller,Michael Jackson
3,The Dark Side Of The Moon,Pink Floyd
4,Rumours,Fleetwood Mac
...,...,...
5670,Into the Hole (At the End of the Light),32crash
5671,"Violin Concerto No. 12 in A Major, Op. 79: And...","Louis Spohr, Takako Nishizaki, Bratislava Phil..."
5672,The Shadows,Necrophobic
5673,Have Yourself A Merry Little Christmas,Ellis Marsalis


In [154]:
not_hot_songs_all_df = pd.concat([not_onehot_songs_new.reset_index(drop=True).iloc[:3000,:],final_df],axis = 1)

In [157]:
not_hot_songs_all_df.to_csv("not_hot_songs_all_df.csv",index=False)

In [156]:
not_hot_songs_all_df = not_hot_songs_all_df[not_hot_songs_all_df['speechiness'] != "Null"]
not_hot_songs_all_df = not_hot_songs_all_df.reset_index(drop=True)
not_hot_songs_all_df.head()

,song_title,artist,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,Random Access Memories,Daft Punk,0.737,0.399,0,-6.337,1,0.305,0.171,0,0.214,0.438,128.019,audio_features,3c1maqNp7lSVWUeOacgVnh,spotify:track:3c1maqNp7lSVWUeOacgVnh,https://api.spotify.com/v1/tracks/3c1maqNp7lSV...,https://api.spotify.com/v1/audio-analysis/3c1m...,244176,4
1,"Good Kid, M.A.A.d City",Kendrick Lamar,0.451,0.831,5,-8.641,0,0.307,0.0623,0,0.185,0.41,176.162,audio_features,5vuJuBqwzHJgCA1ysRfwxZ,spotify:track:5vuJuBqwzHJgCA1ysRfwxZ,https://api.spotify.com/v1/tracks/5vuJuBqwzHJg...,https://api.spotify.com/v1/audio-analysis/5vuJ...,214120,4
2,Thriller,Michael Jackson,0.773,0.859,11,-4.913,1,0.0747,0.0855,0.00018,0.914,0.813,118.458,audio_features,3S2R0EVwBSAVMd5UMgKTL0,spotify:track:3S2R0EVwBSAVMd5UMgKTL0,https://api.spotify.com/v1/tracks/3S2R0EVwBSAV...,https://api.spotify.com/v1/audio-analysis/3S2R...,357267,4
3,The Dark Side Of The Moon,Pink Floyd,0.238,0.943,9,-6.308,1,0.13,0.000449,0.000004,0.102,0.181,113.419,audio_features,4dxT3byfL0g1mlPjHxk18d,spotify:track:4dxT3byfL0g1mlPjHxk18d,https://api.spotify.com/v1/tracks/4dxT3byfL0g1...,https://api.spotify.com/v1/audio-analysis/4dxT...,248613,4
4,Rumours,Fleetwood Mac,0.844,0.262,0,-14.08,1,0.0899,0.68,0,0.111,0.274,104.945,audio_features,4PKZmIXiIt54OQo1E51aTd,spotify:track:4PKZmIXiIt54OQo1E51aTd,https://api.spotify.com/v1/tracks/4PKZmIXiIt54...,https://api.spotify.com/v1/audio-analysis/4PKZ...,151110,4


In [106]:
get_audio_features(not_onehot_100_clean)

Looking for song:  0
Looking for song:  1
Looking for song:  2
Looking for song:  3
Looking for song:  4
Looking for song:  5
Looking for song:  6
Looking for song:  7
Looking for song:  8
Looking for song:  9
Looking for song:  10
Looking for song:  11
Looking for song:  12
Looking for song:  13
Looking for song:  14
Looking for song:  15
Looking for song:  16
Looking for song:  17
Looking for song:  18
Looking for song:  19
Looking for song:  20
Looking for song:  21
Looking for song:  22
Looking for song:  23
Looking for song:  24
Looking for song:  25
Looking for song:  26
Looking for song:  27
Looking for song:  28
Looking for song:  29
Looking for song:  30
Looking for song:  31
Looking for song:  32
Looking for song:  33
Looking for song:  34
Looking for song:  35
Looking for song:  36
Looking for song:  37
Looking for song:  38
Looking for song:  39
Looking for song:  40
Looking for song:  41
Looking for song:  42
Looking for song:  43
Looking for song:  44
Looking for song:  4

Looking for song:  353
Looking for song:  354
Looking for song:  355
Looking for song:  356
Looking for song:  357
Looking for song:  358
Looking for song:  359
Looking for song:  360
Looking for song:  361
Looking for song:  362
Looking for song:  363
Looking for song:  364
Looking for song:  365
Looking for song:  366
Looking for song:  367
Looking for song:  368
Looking for song:  369
Looking for song:  370
Looking for song:  371
Looking for song:  372
Looking for song:  373
Looking for song:  374
Looking for song:  375
Looking for song:  376
Looking for song:  377
Looking for song:  378
Looking for song:  379
Looking for song:  380
Looking for song:  381
Looking for song:  382
Looking for song:  383
Looking for song:  384
Looking for song:  385
Looking for song:  386
Looking for song:  387
Looking for song:  388
Looking for song:  389
Looking for song:  390
Looking for song:  391
Looking for song:  392
Looking for song:  393
Looking for song:  394
Looking for song:  395
Looking for

Looking for song:  708
Looking for song:  709
Looking for song:  710
Looking for song:  711
Looking for song:  712
Looking for song:  713
Looking for song:  714
Looking for song:  715
Looking for song:  716
Looking for song:  717
Looking for song:  718
Looking for song:  719
Looking for song:  720
Looking for song:  721
Looking for song:  722
Looking for song:  723
Looking for song:  724
Looking for song:  725
Looking for song:  726
Looking for song:  727
Looking for song:  728
Looking for song:  729
Looking for song:  730
Looking for song:  731
Looking for song:  732
Looking for song:  733
Looking for song:  734
Looking for song:  735
Looking for song:  736
Looking for song:  737
Looking for song:  738
Looking for song:  739
Looking for song:  740
Looking for song:  741
Looking for song:  742
Looking for song:  743
Looking for song:  744
Looking for song:  745
Looking for song:  746
Looking for song:  747
Looking for song:  748
Looking for song:  749
Looking for song:  750
Looking for

Looking for song:  43
Looking for song:  44
Looking for song:  45
Looking for song:  46
Looking for song:  47
Looking for song:  48
Looking for song:  49
Looking for song:  50
The {} has not been found Fugazi (7 Songs)
Looking for song:  51
Looking for song:  52
Looking for song:  53
Looking for song:  54
Looking for song:  55
Looking for song:  56
Looking for song:  57
Looking for song:  58
Looking for song:  59
Looking for song:  60
Looking for song:  61
Looking for song:  62
Looking for song:  63
Looking for song:  64
Looking for song:  65
Looking for song:  66
Looking for song:  67
Looking for song:  68
Looking for song:  69
Looking for song:  70
Looking for song:  71
Looking for song:  72
Looking for song:  73
Looking for song:  74
Looking for song:  75
Looking for song:  76
Looking for song:  77
Looking for song:  78
Looking for song:  79
Looking for song:  80
The {} has not been found 4630 Bochum
Looking for song:  81
Looking for song:  82
Looking for song:  83
Looking for song:

Looking for song:  356
Looking for song:  357
Looking for song:  358
Looking for song:  359
Looking for song:  360
Looking for song:  361
Looking for song:  362
Looking for song:  363
Looking for song:  364
Looking for song:  365
Looking for song:  366
Looking for song:  367
Looking for song:  368
Looking for song:  369
Looking for song:  370
Looking for song:  371
Looking for song:  372
Looking for song:  373
Looking for song:  374
Looking for song:  375
Looking for song:  376
Looking for song:  377
Looking for song:  378
Looking for song:  379
Looking for song:  380
Looking for song:  381
Looking for song:  382
Looking for song:  383
Looking for song:  384
Looking for song:  385
Looking for song:  386
Looking for song:  387
Looking for song:  388
Looking for song:  389
Looking for song:  390
The {} has not been found Otis Blue / Otis Redding Sings Soul
Looking for song:  391
Looking for song:  392
Looking for song:  393
Looking for song:  394
Looking for song:  395
Looking for song: 

...

Looking for song:  295
Looking for song:  296
Looking for song:  297
Looking for song:  298
Looking for song:  299
Looking for song:  300
Looking for song:  301
Looking for song:  302
Looking for song:  303
Looking for song:  304
Looking for song:  305
Looking for song:  306
Looking for song:  307
Looking for song:  308
Looking for song:  309
Looking for song:  310
The {} has not been found Tear Trax (Instrumental) [Deluxe Edition]
Looking for song:  311
Looking for song:  312
Looking for song:  313
Looking for song:  314
Looking for song:  315
Looking for song:  316
Looking for song:  317
Looking for song:  318
Looking for song:  319
Looking for song:  320
Looking for song:  321
Looking for song:  322
Looking for song:  323
Looking for song:  324
Looking for song:  325
Looking for song:  326
Looking for song:  327
Looking for song:  328
Looking for song:  329
Looking for song:  330
Looking for song:  331
Looking for song:  332
Looking for song:  333
Looking for song:  334
Looking for 

Looking for song:  638
Looking for song:  639
Looking for song:  640
Looking for song:  641
Looking for song:  642
Looking for song:  643
Looking for song:  644
Looking for song:  645
Looking for song:  646
Looking for song:  647
Looking for song:  648
Looking for song:  649
Looking for song:  650
Looking for song:  651
Looking for song:  652
Looking for song:  653
Looking for song:  654
Looking for song:  655
Looking for song:  656
Looking for song:  657
Looking for song:  658
Looking for song:  659
Looking for song:  660
Looking for song:  661
Looking for song:  662
Looking for song:  663
Looking for song:  664
Looking for song:  665
Looking for song:  666
Looking for song:  667
Looking for song:  668
Looking for song:  669
Looking for song:  670
Looking for song:  671
Looking for song:  672
Looking for song:  673
Looking for song:  674
Looking for song:  675
Looking for song:  676
Looking for song:  677
Looking for song:  678
Looking for song:  679
Looking for song:  680
Looking for

Looking for song:  986
Looking for song:  987
Looking for song:  988
Looking for song:  989
Looking for song:  990
Looking for song:  991
Looking for song:  992
Looking for song:  993
Looking for song:  994
Looking for song:  995
Looking for song:  996
Looking for song:  997
Looking for song:  998
Looking for song:  999
Getting audio features
Created file:  audio_feats_4.pickle
Sleeping for 10 seconds
Looking for song:  0
Looking for song:  1
Looking for song:  2
Looking for song:  3
Looking for song:  4
Looking for song:  5
Looking for song:  6
Looking for song:  7
Looking for song:  8
Looking for song:  9
Looking for song:  10
Looking for song:  11
Looking for song:  12
Looking for song:  13
Looking for song:  14
Looking for song:  15
Looking for song:  16
Looking for song:  17
Looking for song:  18
Looking for song:  19
Looking for song:  20
Looking for song:  21
Looking for song:  22
Looking for song:  23
Looking for song:  24
Looking for song:  25
Looking for song:  26
Looking for

Looking for song:  324
Looking for song:  325
Looking for song:  326
Looking for song:  327
Looking for song:  328
Looking for song:  329
Looking for song:  330
Looking for song:  331
Looking for song:  332
Looking for song:  333
Looking for song:  334
Looking for song:  335
Looking for song:  336
Looking for song:  337
Looking for song:  338
Looking for song:  339
Looking for song:  340
Looking for song:  341
Looking for song:  342
Looking for song:  343
Looking for song:  344
Looking for song:  345
Looking for song:  346
Looking for song:  347
Looking for song:  348
Looking for song:  349
Looking for song:  350
Looking for song:  351
Looking for song:  352
Looking for song:  353
Looking for song:  354
Looking for song:  355
Looking for song:  356
Looking for song:  357
Looking for song:  358
Looking for song:  359
Looking for song:  360
Looking for song:  361
Looking for song:  362
Looking for song:  363
Looking for song:  364
Looking for song:  365
Looking for song:  366
Looking for

Looking for song:  638
Looking for song:  639
Looking for song:  640
Looking for song:  641
Looking for song:  642
Looking for song:  643
Looking for song:  644
Looking for song:  645
Looking for song:  646
Looking for song:  647
Looking for song:  648
Looking for song:  649
Looking for song:  650
Looking for song:  651
Looking for song:  652
Looking for song:  653
Looking for song:  654
Looking for song:  655
Looking for song:  656
Looking for song:  657
Looking for song:  658
Looking for song:  659
Looking for song:  660
Looking for song:  661
Looking for song:  662
Looking for song:  663
Looking for song:  664
Looking for song:  665
Looking for song:  666
Looking for song:  667
Looking for song:  668
Looking for song:  669
Looking for song:  670
Looking for song:  671
Looking for song:  672
Looking for song:  673
Looking for song:  674
Getting audio features
Created file:  audio_feats_5.pickle
Sleeping for 10 seconds


In [107]:
!ls

Web_scraping_Lab1.ipynb
__pycache__
audio_feats_0.pickle
audio_feats_1.pickle
audio_feats_2.pickle
audio_feats_3.pickle
audio_feats_4.pickle
audio_feats_5.pickle
audio_feats_one_hot100_songs.pkl
audio_features_one_hot100_songs_df.csv
audio_features_one_hot100_songs_df.csv",index=False
case-study-gnod.md
config.py
discogs_not_hot_songs.csv
hot100.csv
not_hot_100_notebook.ipynb
not_onehot_songs.csv
not_onehot_songs_new.csv
one_hot100_songs_ids.pkl
tracks_features.csv
using_spotipy_project_continued.ipynb


4. Replace the old internal files of songs (hot and not hot) with the extended data frames with the audio features and save them into separate files on the disk.

In [130]:
audio_features_one_hot100_songs_df=pd.read_csv('/Users/devirughani/Desktop/IronHack/Week_6/Spotify_Midterm_Project/data/audio_features_one_hot100_songs_df.csv')
audio_features_one_hot100_songs_df



hot_songs_and_audio_features=pd.concat([one_hot100_clean, audio_features_one_hot100_songs_df], axis=1)
hot_songs_and_audio_features.to_csv('hot_songs_and_audio_features.csv', index=False)